In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
from tensorflow.keras import layers as kl

In [2]:
distr_layer = tfp.layers.DistributionLambda(lambda t: tfd.NegativeBinomial(
        total_count = tf.math.softplus(t[... , 0]), logits = t[..., 1]    
       ))

In [3]:
model = tf.keras.models.Sequential([
    
    kl.Dense(2),
       
    distr_layer
    
])
negloglik = lambda x, rv_x: tf.reduce_mean(-rv_x.log_prob(x))
model.compile(optimizer='adam', loss=negloglik)

In [4]:
model(tf.constant([[1,2,3],[-1,0,1]]))

<tfp.distributions.NegativeBinomial 'sequential/distribution_lambda/NegativeBinomial/' batch_shape=[2] event_shape=[] dtype=float32>

In [5]:
model.weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.14176834, -0.6824337 ],
        [-0.5987934 ,  0.39817536],
        [ 0.18142378,  0.7412889 ]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [6]:
d = tfd.Normal(0, 1)
d.sample()

<tf.Tensor: id=82, shape=(), dtype=float32, numpy=0.36463216>

In [7]:
N = 10000
coefs = np.array([[1, 2, 3],[3, 2, 1]]).T
bias = np.array([3, 0])
X = tfd.Normal(1, 1).sample(sample_shape = [N, 3])

In [8]:
X

<tf.Tensor: id=104, shape=(10000, 3), dtype=float32, numpy=
array([[ 0.5539832 , -0.8368081 ,  2.6849008 ],
       [ 0.87797767, -0.7828914 ,  0.65795726],
       [ 1.301696  ,  1.3853161 ,  2.053258  ],
       ...,
       [ 3.0525506 , -0.31443965,  1.8236    ],
       [ 0.5653211 ,  0.12393051,  1.8508487 ],
       [ 1.1416686 ,  2.1904838 ,  2.3637786 ]], dtype=float32)>

In [9]:
true_neg_bin_params = X @ coefs + bias

In [10]:
true_neg_bin_params

<tf.Tensor: id=108, shape=(10000, 2), dtype=float32, numpy=
array([[ 9.935069 ,  2.6732342],
       [ 4.2860665,  1.7261076],
       [13.232101 ,  8.728978 ],
       ...,
       [10.894471 , 10.352372 ],
       [ 9.365728 ,  3.794673 ],
       [15.613972 , 10.169752 ]], dtype=float32)>

In [11]:
true_distr = distr_layer(true_neg_bin_params)

In [12]:
Y = true_distr.sample()

In [34]:
# old_weights = model.get_weights()
# model.set_weights([coefs, bias])
# model_distr = model(X)
# print(model.evaluate(X, Y, batch_size=100000, verbose=0))
# model.set_weights(old_weights)

917772.75


In [35]:
model.set_weights([coefs, bias])

In [13]:
np.mean(negloglik(Y, true_distr))

8.614815

In [15]:
np.mean(negloglik(Y, model(X)))

2227965.2

In [13]:
print(model.evaluate(X, Y, batch_size=100000, verbose=0))

5707.3525390625


In [21]:
model_distr.variance()

<tf.Tensor: id=822, shape=(1000,), dtype=float32, numpy=
array([1.21238232e+01, 1.00705922e-01, 7.64667285e+03, 3.18914962e+00,
       1.57489258e+04, 2.03829758e+02, 9.40464878e+00, 1.35461194e-02,
       2.55139014e+03, 5.43421484e-04, 1.48330480e-01, 2.06514938e+02,
       2.50184769e-03, 4.73372488e-08, 8.23575234e+04, 1.32114947e+00,
       6.72494507e+01, 6.96377600e+06, 5.32422920e+07, 5.04576117e-02,
       8.25678329e+01, 1.77305031e+01, 6.16659641e+00, 2.71555638e+00,
       1.32029236e+02, 7.03650434e-03, 3.89921808e+00, 8.87841523e-01,
       1.82209956e-03, 1.01331357e+04, 2.46027417e+03, 1.63398814e+00,
       3.79049349e+00, 6.69985504e+01, 4.67193685e-02, 1.36680186e+00,
       1.48160279e+00, 2.11490967e+02, 1.00911297e-01, 2.24391068e-03,
       2.54644629e+03, 7.50159025e-01, 4.95267175e-02, 3.22795331e-01,
       3.65319276e+00, 6.21352673e+00, 3.28687575e+06, 2.36122292e-02,
       4.89925499e+01, 4.10976142e-01, 3.38280737e-01, 4.58471295e-05,
       2.90678040e+0

In [22]:
true_distr.mean()

<tf.Tensor: id=824, shape=(1000,), dtype=float32, numpy=
array([5.39192915e+00, 9.40134972e-02, 2.61385040e+02, 2.01179886e+00,
       1.83725555e+02, 2.39086666e+01, 1.71441627e+00, 1.30386744e-02,
       8.63368378e+01, 5.41332411e-04, 1.41170457e-01, 3.30959282e+01,
       2.48523825e-03, 4.73338169e-08, 8.45050842e+02, 8.71967971e-01,
       2.12845192e+01, 7.70625244e+03, 1.97486152e+04, 4.95060720e-02,
       1.70164757e+01, 6.84960270e+00, 2.24802613e+00, 1.58621025e+00,
       2.24217052e+01, 7.00184191e-03, 2.31625009e+00, 6.30251825e-01,
       1.82117056e-03, 2.63476562e+02, 1.24819069e+02, 1.34678996e+00,
       1.59668016e+00, 1.40467587e+01, 4.60770465e-02, 7.82155454e-01,
       1.05084479e+00, 2.00056953e+01, 7.83467814e-02, 2.16445350e-03,
       1.03910805e+02, 6.58941627e-01, 4.84120101e-02, 1.90160573e-01,
       2.22222304e+00, 2.88203239e+00, 5.04907129e+03, 2.32535619e-02,
       8.42647743e+00, 3.56983066e-01, 3.10838521e-01, 4.55867739e-05,
       1.67614784e+0

In [23]:
model(X).mean()

<tf.Tensor: id=860, shape=(1000,), dtype=float32, numpy=
array([ 1.60564   ,  1.379874  ,  0.37944475,  0.23928322,  0.35129416,
        0.5350127 ,  0.58887446,  3.7094767 ,  0.03705967,  0.6486339 ,
        0.29919487,  3.3640707 ,  0.7692973 ,  0.153582  ,  0.92201734,
        0.36696893,  8.129183  ,  0.6083264 ,  0.49984547,  5.672316  ,
        0.20700514,  0.22036603,  0.12912177,  0.94990593,  0.82027286,
        0.0970009 ,  3.2522144 ,  0.44362554,  0.9279015 ,  0.23899819,
        6.7617755 ,  9.026428  ,  0.7018325 ,  0.28638062,  0.17627268,
        0.74865276,  0.4431658 ,  0.22856337,  0.21052438,  0.15510939,
        0.91958874,  0.3258133 ,  2.0351088 ,  0.12172444,  0.7549416 ,
        0.3646252 ,  0.10030773,  0.71948314,  0.281849  ,  0.5646567 ,
        0.4052561 ,  0.2630552 ,  0.7462153 ,  1.5540676 ,  1.2013553 ,
        1.7074366 ,  1.826355  ,  0.15734006,  0.86663127,  0.48212838,
        2.0446255 ,  0.11625703,  1.1176066 ,  2.1943846 ,  0.33802193,
       

In [24]:
model.evaluate(X, Y)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

2819.313861816406

In [25]:
model.evaluate(X, Y)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

2819.313861816406

In [56]:
true_distr

<tfp.distributions.NegativeBinomial 'distribution_lambda_3_NegativeBinomial' batch_shape=[100000] event_shape=[] dtype=float32>

In [57]:
true_distr.probs_parameter()

<tf.Tensor: id=585, shape=(100000,), dtype=float32, numpy=
array([0.36850506, 0.74067277, 0.66926724, ..., 0.9999394 , 0.32746673,
       0.4854044 ], dtype=float32)>

In [58]:
true_distr.total_count

<tf.Tensor: id=559, shape=(100000,), dtype=float32, numpy=
array([0.2757673 , 0.32079494, 1.5467578 , ..., 8.653196  , 1.9845582 ,
       1.8261846 ], dtype=float32)>

In [59]:
true_distr.logits

<tf.Tensor: id=563, shape=(100000,), dtype=float32, numpy=
array([-0.5386355 ,  1.0494682 ,  0.7048727 , ...,  9.711327  ,
       -0.71966517, -0.05839898], dtype=float32)>

In [61]:
Y

<tf.Tensor: id=606, shape=(100000,), dtype=float32, numpy=
array([0.00000e+00, 2.00000e+00, 1.00000e+00, ..., 1.17367e+05,
       0.00000e+00, 0.00000e+00], dtype=float32)>

In [18]:
model.fit(x = X, y = Y, epochs = 1000, batch_size=128)

Train on 10000 samples
Epoch 1/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 883854.5843
Epoch 2/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 797599.4587
Epoch 3/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 700272.2095
Epoch 4/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 679113.4192
Epoch 5/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 685398.0334
Epoch 6/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 658011.5871
Epoch 7/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 592241.0647
Epoch 8/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 573169.2325
Epoch 9/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 573972.2151
Epoch 10/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 529426.2192
Epoch 11/1000
10000/10000 

10000/10000 [==============================] - 0s 14us/sample - loss: 78979.0719
Epoch 87/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 72332.8899
Epoch 88/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 67804.3402
Epoch 89/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 85219.3225
Epoch 90/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 74885.7351
Epoch 91/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 62060.7935
Epoch 92/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 66371.7472
Epoch 93/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 62596.4001
Epoch 94/1000
10000/10000 [==============================] - 0s 13us/sample - loss: 66457.3843
Epoch 95/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 73813.2194
Epoch 96/1000
10000/10000 [==============================] - 0s 

10000/10000 [==============================] - 0s 14us/sample - loss: 17670.2709
Epoch 172/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 17398.8175
Epoch 173/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 20154.2511
Epoch 174/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 16641.1697
Epoch 175/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 17300.9572
Epoch 176/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 15770.5489
Epoch 177/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 14618.0641
Epoch 178/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 14553.3907
Epoch 179/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 15954.1316
Epoch 180/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 13630.1539
Epoch 181/1000
10000/10000 [===========================

10000/10000 [==============================] - 0s 15us/sample - loss: 3149.0132
Epoch 258/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 3852.9227
Epoch 259/1000
10000/10000 [==============================] - 0s 13us/sample - loss: 2923.3748
Epoch 260/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 3217.9295
Epoch 261/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 3901.5029
Epoch 262/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 3037.5381
Epoch 263/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 3023.1992
Epoch 264/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 3086.9910
Epoch 265/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 2676.1872
Epoch 266/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 2586.8049
Epoch 267/1000
10000/10000 [==============================] - 0s 

10000/10000 [==============================] - 0s 15us/sample - loss: 975.2159
Epoch 344/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 803.2547
Epoch 345/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 722.6273
Epoch 346/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 723.3779
Epoch 347/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 747.4705
Epoch 348/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 797.9753
Epoch 349/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 751.8869
Epoch 350/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 605.0789
Epoch 351/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 786.1795
Epoch 352/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 669.3014
Epoch 353/1000
10000/10000 [==============================] - 0s 14us/sampl

...

10000/10000 [==============================] - 0s 15us/sample - loss: 18.4248
Epoch 606/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 16.6911
Epoch 607/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 18.1171
Epoch 608/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 18.6735
Epoch 609/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 21.8857
Epoch 610/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 17.5065
Epoch 611/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 18.2606
Epoch 612/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 17.4738
Epoch 613/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 16.4658
Epoch 614/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 17.2922
Epoch 615/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 

10000/10000 [==============================] - 0s 14us/sample - loss: 12.3678
Epoch 694/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 12.2775
Epoch 695/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 12.3883
Epoch 696/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 12.2945
Epoch 697/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 12.2846
Epoch 698/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 12.2579
Epoch 699/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 12.4089
Epoch 700/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 12.3997
Epoch 701/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 12.2058
Epoch 702/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 12.3158
Epoch 703/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 

10000/10000 [==============================] - 0s 16us/sample - loss: 11.3733
Epoch 782/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.3541
Epoch 783/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.3564
Epoch 784/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 11.3969
Epoch 785/1000
10000/10000 [==============================] - 0s 20us/sample - loss: 11.3463
Epoch 786/1000
10000/10000 [==============================] - 0s 19us/sample - loss: 11.3028
Epoch 787/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 11.3622
Epoch 788/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.3336
Epoch 789/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.3042
Epoch 790/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 11.3274
Epoch 791/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 

10000/10000 [==============================] - 0s 16us/sample - loss: 11.1683
Epoch 870/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 11.1788
Epoch 871/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 11.1714
Epoch 872/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 11.1714
Epoch 873/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1742
Epoch 874/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1645
Epoch 875/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1657
Epoch 876/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1644
Epoch 877/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1687
Epoch 878/1000
10000/10000 [==============================] - 0s 17us/sample - loss: 11.1750
Epoch 879/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 

10000/10000 [==============================] - 0s 17us/sample - loss: 11.1423
Epoch 958/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1427
Epoch 959/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1437
Epoch 960/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1431
Epoch 961/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 11.1438
Epoch 962/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 11.1375
Epoch 963/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 11.1419
Epoch 964/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 11.1402
Epoch 965/1000
10000/10000 [==============================] - 0s 16us/sample - loss: 11.1402
Epoch 966/1000
10000/10000 [==============================] - 0s 14us/sample - loss: 11.1491
Epoch 967/1000
10000/10000 [==============================] - 0s 15us/sample - loss: 

In [19]:
np.mean(negloglik(Y, model(X)))

11.128087

In [17]:
l = model.loss_functions[0]

In [18]:
l.call(Y, model(X))

<tf.Tensor: id=41183, shape=(), dtype=float32, numpy=6.023795>

In [24]:
dense = model.layers[0]

In [27]:
dense.kernel_regularizer

In [ ]:
kl.Dense()

In [26]:
dense.a


In [20]:
model.weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.03124883,  0.28588724],
        [-0.03150563,  0.2923285 ],
        [-0.03221646,  0.28648335]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(2,) dtype=float32, numpy=array([-2.0391998, 16.384024 ], dtype=float32)>]

In [68]:
model(X).total_count

<tf.Tensor: id=399687, shape=(100000,), dtype=float32, numpy=
array([0.05909398, 0.05949057, 0.06059545, ..., 0.06368538, 0.05943586,
       0.06014879], dtype=float32)>

In [69]:
model(X).probs_parameter()

<tf.Tensor: id=399747, shape=(100000,), dtype=float32, numpy=
array([0.99999475, 0.9999949 , 0.99999505, ..., 0.9999956 , 0.9999945 ,
       0.9999949 ], dtype=float32)>

In [21]:
def get_loss():
    return tf.reduce_mean(negloglik(Y, model(X)))

In [22]:
get_loss()

<tf.Tensor: id=160078, shape=(), dtype=float32, numpy=11.128087>

In [23]:
optimizer = tf.optimizers.Adam()

In [24]:
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = get_loss()

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

In [25]:
for i in range(100000):
    loss = train_step()
    if i % 100 == 0:
        print(loss)

tf.Tensor(11.128087, shape=(), dtype=float32)
tf.Tensor(11.062134, shape=(), dtype=float32)
tf.Tensor(11.052348, shape=(), dtype=float32)
tf.Tensor(11.0327, shape=(), dtype=float32)
tf.Tensor(10.954716, shape=(), dtype=float32)
tf.Tensor(10.967147, shape=(), dtype=float32)
tf.Tensor(10.987995, shape=(), dtype=float32)
tf.Tensor(10.989368, shape=(), dtype=float32)
tf.Tensor(10.980344, shape=(), dtype=float32)
tf.Tensor(10.9544935, shape=(), dtype=float32)
tf.Tensor(10.939382, shape=(), dtype=float32)
tf.Tensor(10.905773, shape=(), dtype=float32)
tf.Tensor(10.9035, shape=(), dtype=float32)
tf.Tensor(10.890143, shape=(), dtype=float32)
tf.Tensor(10.87322, shape=(), dtype=float32)
tf.Tensor(10.866693, shape=(), dtype=float32)
tf.Tensor(10.856206, shape=(), dtype=float32)
tf.Tensor(10.852836, shape=(), dtype=float32)
tf.Tensor(10.847836, shape=(), dtype=float32)
tf.Tensor(10.839115, shape=(), dtype=float32)
tf.Tensor(10.839273, shape=(), dtype=float32)
tf.Tensor(10.837333, shape=(), dtype=f

tf.Tensor(8.560568, shape=(), dtype=float32)
tf.Tensor(8.544261, shape=(), dtype=float32)
tf.Tensor(8.556552, shape=(), dtype=float32)
tf.Tensor(8.558686, shape=(), dtype=float32)
tf.Tensor(8.553261, shape=(), dtype=float32)
tf.Tensor(8.571409, shape=(), dtype=float32)
tf.Tensor(8.556756, shape=(), dtype=float32)
tf.Tensor(8.569316, shape=(), dtype=float32)
tf.Tensor(8.570954, shape=(), dtype=float32)
tf.Tensor(8.575504, shape=(), dtype=float32)
tf.Tensor(8.5664015, shape=(), dtype=float32)
tf.Tensor(8.569334, shape=(), dtype=float32)
tf.Tensor(8.566512, shape=(), dtype=float32)
tf.Tensor(8.562978, shape=(), dtype=float32)
tf.Tensor(8.55217, shape=(), dtype=float32)
tf.Tensor(8.542455, shape=(), dtype=float32)
tf.Tensor(8.541173, shape=(), dtype=float32)
tf.Tensor(8.535672, shape=(), dtype=float32)
tf.Tensor(8.543259, shape=(), dtype=float32)
tf.Tensor(8.563656, shape=(), dtype=float32)
tf.Tensor(8.556761, shape=(), dtype=float32)
tf.Tensor(8.573159, shape=(), dtype=float32)
tf.Tensor(

KeyboardInterrupt: 

In [26]:
model.trainable_weights

[<tf.Variable 'sequential/dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[0.9516669, 3.0018325],
        [1.9121954, 2.0007813],
        [2.794243 , 1.00997  ]], dtype=float32)>,
 <tf.Variable 'sequential/dense/bias:0' shape=(2,) dtype=float32, numpy=array([2.942339  , 0.03384899], dtype=float32)>]